In [1]:
import pytesseract
from PIL import Image
import pandas as pd
import cv2 as cv

In [2]:
# function to do skew correction in images
def deskew(img):
    img = cv2.Canny(img, 100, 200)
    coords = np.column_stack(np.where(img > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = img.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

In [3]:
# function to do image processing on the image
def img_processing(image):
    img = cv2.resize(image, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv.COLOR_BGR2GRAY)
    #kernel = np.ones((1, 1), np.uint8)
    #img = cv2.dilate(img, kernel, iterations=1)
    #img = cv2.erode(img, kernel, iterations=1)
    img = cv.GaussianBlur(img, (9, 9), 0)
    #img = cv.bilateralFilter(img,9,75,75)
    #img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    return img

In [4]:
# function to find text in image
def ocr(image):
    text = pytesseract.image_to_string(image)
    return text

In [5]:
# function to check whether GSTIN is present or not in image
def check_GSTIN(text):
    flag1 = text.find('GSTIN')
    if(flag1 > 0):
        return flag1 +  6
    
    flag2 = text.find('STIN')
    if(flag2 > 0):
        return flag2  + 5
    
    return -1

In [6]:
# function to find GST Number in image
def GST_number(text,pos):
    count = 0
    number = []
    while((text[pos+count].isdigit() == False) and count <10):
        count+=1
    if(text[pos+count].isdigit()):
        if(text[pos+count-1].isalpha()):
            count = count - 1
            number = text[pos+count:pos+count+15]
            
    return number

In [7]:
data = pd.read_csv('gst_labels_detected.csv')
row,col = data.shape

GSTIN_detected = 0

for i in range(row):
    img_name = data.iloc[i]['Name']
    img_location = 'detected/' + img_name
    #image = Image.open(img_location)
    img = cv.imread(img_location)
    
    # Do processing on image
    #processed_img = img_processing(image)
    
    # Find texts on image
    text = ocr(img)
    
    # Check GSTIN is present or not
    flag_GSTIN = check_GSTIN(text)
    if(flag_GSTIN>=0):
        GSTIN_detected += 1
        data.iloc[i]['Label'] = 'GSTIN_Detected'
    else:
        data.iloc[i]['Label'] = 'GSTIN_Not_Detected'
    
    # If GSTIN detected find the number
    #if(flag_GSTIN>=0):
    #    gst_number = GST_number(text,flag_GSTIN)
    #    data.iloc[i]['GST_Number'] = gst_number
    #else:
    #    data.iloc[i]['GST_Number'] = None
    
    # Display Image
    #cv.imshow(img_name,img)
    #cv.imshow('After Processing',processed_img)
    #cv.waitKey(0)
    #cv.destroyAllWindows()

print(GSTIN_detected)

D:\users\vivek\Downloads\Anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\users\vivek\Downloads\Anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


194
